In [11]:
# Notebook: Hugging Face Pre-Built Task Pipelines
# Author: Thomas Purk
# Date: 2025-04-01
# Reference: https://huggingface.co/docs/transformers/v4.50.0/en/main_classes/pipelines

# Hugging Face Pre-Built Task Pipelines

The Pipeline class of the Hugging Face Transformers package can instantiate new pipeline objects using a 'task' string. At this time there are 29 pre-defined pipeline strings covering text, audio, video, and image type tasks.

This notebook investigates common pre-defined task-based pipelines and their default parameters and metadata.

- Sentiment Analysis
- Text Generation
- Mask Filling
- Named Entity Recognition
- Question Answering
- Summarization
- Translation

In [12]:
# Package Installs

#!pip install transformers

!pip list | grep transformers

sentence-transformers              3.4.1
transformers                       4.50.3


In [20]:
# Setup the Notebook

# General
import os
import json
import logging
logging.getLogger("transformers").setLevel(logging.WARNING) # Suppress unnecessary logging

# Visualization
import pprint
from IPython.core.display import display, HTML

# Data, Science, & Math
import numpy as np
import pandas as pd

# NLP
import transformers
from transformers.pipelines import SUPPORTED_TASKS
from transformers import pipeline

In [14]:
def convert_np_types(obj):
    '''Recursively convert NumPy types to standard Python types.
    This function originated from ChatGPT.

    Parameters:
        obj (object): The object to convert.

    Returns:
        object: The converted object.

    '''
    if isinstance(obj, (np.float32, np.float64)):
        return float(obj)  # Convert NumPy float to Python float
    elif isinstance(obj, (np.int32, np.int64)):
        return int(obj)  # Convert NumPy int to Python int
    elif isinstance(obj, dict):
        return {key: convert_np_types(value) for key, value in obj.items()}  # Recurse into dict
    elif isinstance(obj, list):
        return [convert_np_types(item) for item in obj]  # Recurse into list
    return obj  # Return unchanged if it's not a NumPy type



def get_task_metadata(task_name, framework):
    '''Assemble metadata about each predefined pipeline tasks. This is a
    work-around to the get_default_model_and_revision() function which
    returns an error for 'translation task'.

    Parameters:
        task_name (str): An items from transformers.pipeline.SUPPORTED_TASKS.
        framework (str): "pt" for Pytorch or "tf" for TensorFlow

    '''

    meta_dict = {}
    task = SUPPORTED_TASKS[task_name]
    meta_dict['task_name'] = task_name

    # Handle special cases where the default dict is parameratized by language
    if('model'in task['default']):
        model_name, model_revision = task['default']['model'][framework]
        meta_dict['model_name'] = model_name
        meta_dict['model_revision'] = model_revision
    elif(('en','fr') in task['default']):
        meta_dict['model_name'] = 'N/A'
        model_name, model_revision = task['default'][('en','fr')]['model'][framework]
        meta_dict['model_name'] = model_name
        meta_dict['model_revision'] = model_revision

    if(framework in task):
        meta_dict['model_class'] = str(task[framework][0])
    else:
        meta_dict['model_class'] = 'N/A'

    if('type' in task):
        meta_dict['model_type'] = task['type']
    else:
        meta_dict['model_type'] = 'N/A'

    if('impl' in task):
        meta_dict['model_impl'] = task['impl']
    else:
        meta_dict['model_impl'] = 'N/A'

    return meta_dict

# Report on each pre-built task
# Print meta data about each built in task
for task_name in SUPPORTED_TASKS.keys():
    m = get_task_metadata(task_name, 'pt')
    print(f'Task Name: {m["task_name"]}')
    print('-----------------------------')
    print(f'Model Type: {m["model_type"]}')
    print(f'Model Name: {m["model_name"]}')
    print(f'Model Revision: {m["model_revision"]}')
    print(f'Model Class: {m["model_class"]}')
    print(f'Model Implementation: {m["model_impl"]}')
    print()

Task Name: audio-classification
-----------------------------
Model Type: audio
Model Name: superb/wav2vec2-base-superb-ks
Model Revision: 372e048
Model Class: <class 'transformers.models.auto.modeling_auto.AutoModelForAudioClassification'>
Model Implementation: <class 'transformers.pipelines.audio_classification.AudioClassificationPipeline'>

Task Name: automatic-speech-recognition
-----------------------------
Model Type: multimodal
Model Name: facebook/wav2vec2-base-960h
Model Revision: 22aad52
Model Class: <class 'transformers.models.auto.modeling_auto.AutoModelForCTC'>
Model Implementation: <class 'transformers.pipelines.automatic_speech_recognition.AutomaticSpeechRecognitionPipeline'>

Task Name: text-to-audio
-----------------------------
Model Type: text
Model Name: suno/bark-small
Model Revision: 1dbd7a1
Model Class: <class 'transformers.models.auto.modeling_auto.AutoModelForTextToWaveform'>
Model Implementation: <class 'transformers.pipelines.text_to_audio.TextToAudioPipeline

In [15]:
# Example: Default Sentiment Analysis Pipeline
# Classifies text

# Create a Default 'text-classification'/
sa_pipeline = pipeline('sentiment-analysis')

# 'sentiment-analysis' is an alias for 'text-classification'
print('\nMetadata:')
display(get_task_metadata('text-classification', 'pt'))
print()


# Get model configuration
model_config = sa_pipeline.model.config
print(f'Class Labels: {model_config.id2label}')
print()
print(f'Model Card: https://huggingface.co/{model_config._name_or_path}')
print()

inputs = [
    "I absolutely love this new phone! It's fast and the camera is amazing.",
    "The service at the restaurant was terrible, and the food was cold when it arrived.",
    "I'm feeling pretty neutral about the movie—it had some good moments but was mostly forgettable.",
    "Winning the competition was the best experience of my life!",
    "I can't believe how frustrating this software update is; nothing works properly now."
]

# Execute the sentiment analysis model
results = sa_pipeline(inputs=inputs)

df = pd.DataFrame(results)
df['inputs'] = inputs
df

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu



Metadata:


{'task_name': 'text-classification',
 'model_name': 'distilbert/distilbert-base-uncased-finetuned-sst-2-english',
 'model_revision': '714eb0f',
 'model_class': "<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>",
 'model_type': 'text',
 'model_impl': transformers.pipelines.text_classification.TextClassificationPipeline}


Class Labels: {0: 'NEGATIVE', 1: 'POSITIVE'}

Model Card: https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english



,label,score,inputs
0,POSITIVE,0.999887,I absolutely love this new phone! It's fast an...
1,NEGATIVE,0.999531,"The service at the restaurant was terrible, an..."
2,NEGATIVE,0.999625,I'm feeling pretty neutral about the movie—it ...
3,POSITIVE,0.999847,Winning the competition was the best experienc...
4,NEGATIVE,0.999699,I can't believe how frustrating this software ...


In [16]:
# Example: Default Text Generation Pipeline
# Classifies text

# Create a Default 'text-classification'/
tg_pipeline = pipeline(
    task='text-generation'
)

# 'sentiment-analysis' is an alias for 'text-classification'
print('\nMetadata:')
display(get_task_metadata('text-generation', 'pt'))
print()


# Get model configuration
model_config = tg_pipeline.model.config
print(f'Class Labels: {model_config.id2label}')
print()
print(f'Model Card: https://huggingface.co/{model_config._name_or_path}')
print()

inputs = [
    "I absolutely love this new phone! It's fast and the camera is amazing.",
    "The service at the restaurant was terrible, and the food was cold when it arrived.",
    "I'm feeling pretty neutral about the movie—it had some good moments but was mostly forgettable.",
    "Winning the competition was the best experience of my life!",
    "I can't believe how frustrating this software update is; nothing works properly now."
]

# Execute the text generation model
results = tg_pipeline(text_inputs=inputs)

# Formate the resuls
results = [x[0]['generated_text'] for x in results]

# Create a table of inputs and results
df = pd.DataFrame(results, columns=['results'])
df['inputs'] = inputs

for i, r in df.iterrows():
    print(f'\nInput:')
    display(r["inputs"])
    print(f'\nGenerated Text: ')
    display(r["results"])
    print('\n---------------------------')

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu



Metadata:


{'task_name': 'text-generation',
 'model_name': 'openai-community/gpt2',
 'model_revision': '607a30d',
 'model_class': "<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>",
 'model_type': 'text',
 'model_impl': transformers.pipelines.text_generation.TextGenerationPipeline}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Class Labels: {0: 'LABEL_0', 1: 'LABEL_1'}

Model Card: https://huggingface.co/openai-community/gpt2



...


---------------------------

Input:


"I can't believe how frustrating this software update is; nothing works properly now."


Generated Text: 


"I can't believe how frustrating this software update is; nothing works properly now. And yet I can use the update and it works fine. If I have problems with the phone on any other network I would suggest you install Nexus 6P first as the"


---------------------------


In [17]:
# Example: Default Text Generation Pipeline, But Specify the Model
# Classifies text

# Create a Default 'text-generation'
tg_pipeline = pipeline(
    task='text-generation',
    model='distilgpt2'
)

# 'sentiment-analysis' is an alias for 'text-classification'
print('\nMetadata:')
display(get_task_metadata('text-generation', 'pt'))
print()


# Get model configuration
model_config = tg_pipeline.model.config
print(f'Class Labels: {model_config.id2label}')
print()
print(f'Model Card: https://huggingface.co/{model_config._name_or_path}')
print()

inputs = [
    "I absolutely love this new phone! It's fast and the camera is amazing.",
    "The service at the restaurant was terrible, and the food was cold when it arrived.",
    "I'm feeling pretty neutral about the movie—it had some good moments but was mostly forgettable.",
    "Winning the competition was the best experience of my life!",
    "I can't believe how frustrating this software update is; nothing works properly now."
]

# Execute the text generation model
results = tg_pipeline(text_inputs=inputs)

# Formate the resuls
results = [x[0]['generated_text'] for x in results]

# Create a table of inputs and results
df = pd.DataFrame(results, columns=['results'])
df['inputs'] = inputs

for i, r in df.iterrows():
    print(f'\nInput:')
    display(r["inputs"])
    print(f'\nGenerated Text: ')
    display(r["results"])
    print('\n---------------------------')

Device set to use cpu



Metadata:


{'task_name': 'text-generation',
 'model_name': 'openai-community/gpt2',
 'model_revision': '607a30d',
 'model_class': "<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>",
 'model_type': 'text',
 'model_impl': transformers.pipelines.text_generation.TextGenerationPipeline}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Class Labels: {0: 'LABEL_0'}

Model Card: https://huggingface.co/distilgpt2



...


---------------------------

Input:


"I can't believe how frustrating this software update is; nothing works properly now."


Generated Text: 


"I can't believe how frustrating this software update is; nothing works properly now.\n\n\nI have to admit that it's nice that this can be downloaded on Windows 10 and work on macOS 10.10 (which I have no doubt supports some"


---------------------------


In [18]:
# Example: Default Fill Mask Pipeline
# Classifies text

# Create a Default 'text-classification'
fm_pipeline = pipeline(
    task='fill-mask',
)

# 'sentiment-analysis' is an alias for 'text-classification'
print('\nMetadata:')
display(get_task_metadata('fill-mask', 'pt'))
print()


# Get model configuration
model_config = fm_pipeline.model.config
print(f'Class Labels: {model_config.id2label}')
print()
print(f'Model Card: https://huggingface.co/{model_config._name_or_path}')
print()

inputs = [
    "I absolutely love this <mask>! It's fast and the camera is <mask>.",
    "The service at the restaurant was <mask>, and the food was <mask> when it arrived.",
    "I'm feeling pretty <mask> about the movie—it had some good moments but was mostly <mask>.",
    "Winning the <mask> was the best experience of my life!",
    "I can't believe how frustrating this <mask> update is; nothing works properly now."
]


# Execute the mask fill model
results = fm_pipeline(
    inputs=inputs,
    top_k=2
)

# Format # Print the results
print(json.dumps(convert_np_types(results), indent=4))

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu



Metadata:


{'task_name': 'fill-mask',
 'model_name': 'distilbert/distilroberta-base',
 'model_revision': 'fb53ab8',
 'model_class': "<class 'transformers.models.auto.modeling_auto.AutoModelForMaskedLM'>",
 'model_type': 'text',
 'model_impl': transformers.pipelines.fill_mask.FillMaskPipeline}


Class Labels: {0: 'LABEL_0', 1: 'LABEL_1'}

Model Card: https://huggingface.co/distilbert/distilroberta-base

[
    [
        [
            {
                "score": 0.3331633508205414,
                "token": 2280,
                "token_str": " camera",
                "sequence": "<s>I absolutely love this camera! It's fast and the camera is<mask>.</s>"
            },
            {
                "score": 0.03793039172887802,
                "token": 1028,
                "token_str": " phone",
                "sequence": "<s>I absolutely love this phone! It's fast and the camera is<mask>.</s>"
            }
        ],
        [
            {
                "score": 0.12083110958337784,
                "token": 12058,
                "token_str": " gorgeous",
                "sequence": "<s>I absolutely love this<mask>! It's fast and the camera is gorgeous.</s>"
            },
            {
                "score": 0.11521915346384048,
                "token": 4406,
           

In [19]:
# Example: Default Named Entity Recognition
# Classifies tokens
# NOTE: 'ner' is an alias for 'token-classification'

# Create a Default 'ner' pipeline
ner_pipeline = pipeline(
    task='ner',
)

# 'sentiment-analysis' is an alias for 'text-classification'
print('\nMetadata:')
display(get_task_metadata('token-classification', 'pt'))
print()


# Get model configuration
model_config = ner_pipeline.model.config
print(f'Class Labels: {model_config.id2label}')
print()
print(f'Model Card: https://huggingface.co/{model_config._name_or_path}')
print()

inputs = [
    "The Titanic sank in the Atlantic Ocean in 1912.",
    "Pablo Picasso was a Spanish painter and sculptor known for co-founding Cubism.",
    "J.K. Rowling wrote the Harry Potter series, starting with 'Harry Potter and the Sorcerer’s Stone'.",
    "The Great Wall of China is one of the Seven Wonders of the World.",
    "The Oscars, also known as the Academy Awards, recognize excellence in the film industry.",

]


# Execute the token classification model
results = ner_pipeline(
    inputs=inputs,
    grouped_entities=True # Re-assembles "Pablo" & "Pacaso" tokens
)

# Format # Print the results
print(json.dumps(convert_np_types(results), indent=4))

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu



Metadata:


{'task_name': 'token-classification',
 'model_name': 'dbmdz/bert-large-cased-finetuned-conll03-english',
 'model_revision': '4c53496',
 'model_class': "<class 'transformers.models.auto.modeling_auto.AutoModelForTokenClassification'>",
 'model_type': 'text',
 'model_impl': transformers.pipelines.token_classification.TokenClassificationPipeline}

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(



Class Labels: {0: 'O', 1: 'B-MISC', 2: 'I-MISC', 3: 'B-PER', 4: 'I-PER', 5: 'B-ORG', 6: 'I-ORG', 7: 'B-LOC', 8: 'I-LOC'}

Model Card: https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english

[
    [
        {
            "entity_group": "MISC",
            "score": 0.8621680736541748,
            "word": "Titanic",
            "start": 4,
            "end": 11
        },
        {
            "entity_group": "LOC",
            "score": 0.9976879358291626,
            "word": "Atlantic Ocean",
            "start": 24,
            "end": 38
        }
    ],
    [
        {
            "entity_group": "PER",
            "score": 0.9983973503112793,
            "word": "Pablo Picasso",
            "start": 0,
            "end": 13
        },
        {
            "entity_group": "MISC",
            "score": 0.9976692795753479,
            "word": "Spanish",
            "start": 20,
            "end": 27
        },
        {
            "entity_group": "MISC",
            "

In [21]:
# Example: Default Summarization Pipeline
# Classifies text

# Create a Default 'summarization'
sum_pipeline = pipeline(
    task='summarization',
)

print('\nMetadata:')
display(get_task_metadata('summarization', 'pt'))
print()


# Get model configuration
model_config = sum_pipeline.model.config
print(f'Class Labels: {model_config.id2label}')
print()
print(f'Model Card: https://huggingface.co/{model_config._name_or_path}')
print()

inputs = '''Traditional hand tool woodworking is a craft that emphasizes
precision, skill, and patience. Unlike modern power tools, hand tools such as
chisels, hand planes, and saws allow artisans to shape wood with a high level
of control. This method of woodworking has been practiced for centuries, relying
 on techniques passed down through generations to create furniture, cabinetry,
 and intricate wood carvings.

One of the key benefits of working with hand tools is the ability to produce
fine, detailed joinery without the noise and dust associated with power tools.
Techniques like dovetail joints, mortise and tenon connections, and hand-planed
surfaces showcase the craftsmanship involved. Woodworkers often select tools
based on the type of wood they are working with, ensuring smooth cuts and
precise fittings.

In addition to its practical advantages, hand tool woodworking fosters a deep
connection between the artisan and the material. The rhythmic motion of a hand
saw or the delicate shaving of a plane allows for a meditative experience that
many woodworkers find rewarding. As more people seek sustainable and traditional
 crafting methods, hand tool woodworking continues to thrive, preserving
 historical skills while promoting an appreciation for high-quality, handmade
 pieces.
'''


# Execute the text generation model
results = sum_pipeline(inputs)
results = results[0]['summary_text']

# Format # Print the
print('Result:')
display(HTML(f"<pre style='white-space: pre-wrap; word-wrap: break-word;'>{results}</pre>"))

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu



Metadata:


{'task_name': 'summarization',
 'model_name': 'sshleifer/distilbart-cnn-12-6',
 'model_revision': 'a4f8f3e',
 'model_class': "<class 'transformers.models.auto.modeling_auto.AutoModelForSeq2SeqLM'>",
 'model_type': 'text',
 'model_impl': transformers.pipelines.text2text_generation.SummarizationPipeline}


Class Labels: {0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2'}

Model Card: https://huggingface.co/sshleifer/distilbart-cnn-12-6

Result:


In [22]:
# Observation: The summary is ALMOST there, but only makes sense under great effort
# but the reader.

In [23]:
# Example: Default Question Answering
# Classifies tokens


# Create a Default 'question-answering' pipeline
qa_pipeline = pipeline(
    task='question-answering'
)

print('\nMetadata:')
display(get_task_metadata('question-answering', 'pt'))
print()


# Get model configuration
model_config = qa_pipeline.model.config
print(f'Class Labels: {model_config.id2label}')
print()
print(f'Model Card: https://huggingface.co/{model_config._name_or_path}')
print()

# Use the three paragraphs about handtool woodworking defined above.
context = inputs

question = "What are some tools used in hand tool woodworking?"

# Execute the text generation model
results = qa_pipeline(
    question=question,
    context=context
)

# Format # Print the results
print(json.dumps(convert_np_types(results), indent=4))

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu



Metadata:


{'task_name': 'question-answering',
 'model_name': 'distilbert/distilbert-base-cased-distilled-squad',
 'model_revision': '564e9b5',
 'model_class': "<class 'transformers.models.auto.modeling_auto.AutoModelForQuestionAnswering'>",
 'model_type': 'text',
 'model_impl': transformers.pipelines.question_answering.QuestionAnsweringPipeline}


Class Labels: {0: 'LABEL_0', 1: 'LABEL_1'}

Model Card: https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad

{
    "score": 0.9732940793037415,
    "start": 139,
    "end": 169,
    "answer": "chisels, hand planes, and saws"
}


In [24]:
# Example: Default Translation, with specific model
# Classifies tokens


# Create a Default 'translation' pipeline
tr_pipeline = pipeline(
    task='translation',
    model="Helsinki-NLP/opus-mt-fr-en"
)

print('\nMetadata:')
display(get_task_metadata('translation', 'pt'))
print()


# Get model configuration
model_config = tr_pipeline.model.config
print(f'Class Labels: {model_config.id2label}')
print()
print(f'Model Card: https://huggingface.co/{model_config._name_or_path}')
print()

inputs = [
    "Bonjour, comment allez-vous ?",
    "Il fait beau aujourd'hui.",
    "Je voudrais un café, s'il vous plaît.",
    "Où est la gare ?",
    "Merci, au revoir."
]

'''English Reference

    "Hello, how are you?",
    "The weather is nice today.",
    "I would like a coffee, please.",
    "Where is the train station?",
    "Thank you, goodbye."
'''


# Execute the text generation model
results = tr_pipeline(inputs)

# Format # Print the results
print(json.dumps(convert_np_types(results), indent=4))

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu



Metadata:


{'task_name': 'translation',
 'model_name': 'google-t5/t5-base',
 'model_revision': 'a9723ea',
 'model_class': "<class 'transformers.models.auto.modeling_auto.AutoModelForSeq2SeqLM'>",
 'model_type': 'text',
 'model_impl': transformers.pipelines.text2text_generation.TranslationPipeline}


Class Labels: {0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2'}

Model Card: https://huggingface.co/Helsinki-NLP/opus-mt-fr-en

[
    {
        "translation_text": "Hello, how are you?"
    },
    {
        "translation_text": "It's beautiful today."
    },
    {
        "translation_text": "I'd like some coffee, please."
    },
    {
        "translation_text": "Where's the station?"
    },
    {
        "translation_text": "Thank you, bye."
    }
]
